In [1]:
#CELDA 1: Imports, configuraciones y versiones

import numpy as np
import pandas as pd
import joblib
from pathlib import Path

%run 00_setup.ipynb

#RUTAS
from src.config.rutas import BASE_DIR

MODELS_DIR = BASE_DIR / "models"
VERSION = "v2"

tfidf = joblib.load(MODELS_DIR / "tfidf_vect_v2_robusto.joblib")
svm = joblib.load(MODELS_DIR / "modelo_svm_v2_robusto.joblib")

In [2]:
#CELDA 2: Acceso al SVM base calibrado

if hasattr(svm, "calibrated_classifiers_"):             #Se comprueba si el objeto svm contiene el atributo para saber si SVM ha sido calibrado
    coefs = np.array([                                  #Se extraen los coeficientes de cada SVM entrenado con cada fold de validacion cruzada
        c.estimator.coef_[0]                            #Cada c.estimator es el SVM real y coef[0] son los pesos de la features al ser clasificacion binaria
        for c in svm.calibrated_classifiers_            #Lista de clasificadores entrenados, siendo uno por cad fold
    ])
    coef = coefs.mean(axis=0)                           #Se promedia los coeficientes de los folds verticalmente para obtener el vector de pesos representativo por palabra
    print(f"SVM calibrado con ({coefs.shape[0]}) folds promediados")
else:
    coef = svm.coef_[0]                                 #Si el SVM no esta calibrado se devuelve el unico modelo y sus coeficientes
    print("SVM no calibrado")


#se recuperan los tokens del vectorizador, estos token coinciden ordenadamente con los pesos del SVM
feat_names = tfidf.get_feature_names_out()
print(f"Num de features: {len(feat_names)}")

SVM calibrado con (3) folds promediados
Num de features: 33687


In [3]:
#CELDA 3: Tokens mas indicativos de muestras phishing

TOP_K = 20

index_top_phishing = np.argsort(coef)[-TOP_K:]      #Se obtiene los ultimos 20 indices de los coeficientes mas altos, eso se hace al devolver posciones de los indices que ordenarian coef de - a +
top_phishing = pd.DataFrame({                       #Se contruye el DataFrame a partir de los indices al convertirlos en palabras y pesos reales
    "token": feat_names[index_top_phishing],
    "peso": coef[index_top_phishing]
}).sort_values("peso", ascending=False)             #Se ordena por peso con la palabra mas influyente arriba y la que menos abajo

top_phishing

,token,peso
19,http,3.938328
18,https,3.317727
17,com,1.779054
16,top,1.661142
15,rd,1.518756
14,txt,1.479135
13,you have,1.471024
12,text,1.445981
11,php,1.428465
10,free,1.361225


In [4]:
#CELDA 4: Tokens mas indicativos de muestras legitimas

index_top_legitimas = np.argsort(coef)[:TOP_K]      #Se obtiene los primeros 20 indices de los coeficientes mas bajos, eso se hace al devolver posciones de los indices que ordenarian coef de - a +
top_legitimas = pd.DataFrame({                      #Lo mismo que en la celda anterior pero con la variacion que se ordena ascendentemente, es decir, los valores de peso mas negativos arriba
    "token": feat_names[index_top_legitimas],
    "peso": coef[index_top_legitimas]
}).sort_values("peso", ascending=True)

top_legitimas

,token,peso
0,https www,-4.844522
1,enron,-2.331149
2,wrote,-2.207437
3,thanks,-1.758824
4,pm,-1.549222
5,ierant,-1.269357
6,my,-1.259247
7,university,-1.238001
8,that,-1.159726
9,ok,-1.154503


In [5]:
#CELDA 5: Guardamos resultados

top_phishing.to_csv(f"../models/top_tokens_phishing_robusto_{VERSION}.csv", index=False)
top_legitimas.to_csv(f"../models/top_tokens_legitimos_robusto_{VERSION}.csv", index=False)

print("CSV de explicabilidad guardados en /models")

CSV de explicabilidad guardados en /models
